In [2]:
:dep rand = "0.8.5"
:dep micrograd_rs = { path = "." }

use std::iter::zip;
use micrograd_rs::engine::Value;
use micrograd_rs::nn::{ActFunc, MLP, Module};

In [3]:
// Inputs.
let xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
];
// Desired targets.
let ys = [1.0, -1.0, -1.0, 1.0];

// Convert inputs of f64 to Value.
let inputs: Vec<Vec<Value>> = xs.iter().map(|xrow| vec![Value::new(xrow[0]), Value::new(xrow[1]), Value::new(xrow[2])]).collect();

// MLP with three inputs, two 4-size layers, and single output.
let mlp = MLP::new(3, &[4, 4, 1], ActFunc::TanH);

let mut ypred: Vec<Value> = Vec::new();
for _ in 0..20 {
    // Forward pass.
    ypred = Vec::new();
    for x in &inputs {
        ypred.push(mlp.forward(x)[0].clone());
    }
    let loss = zip(ys, ypred.iter())
        .map(|(ygt, yout)| (yout - ygt).pow(2.0))
        .fold(Value::new(0.0), |a, b| a + b);

    // Backward pass. Don't forget to reset grads.
    mlp.zero_grad();
    loss.backward();

    // Update.
    for p in mlp.parameters() {
        p.set_data(p.get_data() + (-0.1 * p.get_grad()));
    }
};

// Values data should be close to [1.0, -1.0, -1.0, 1.0].
println!("{:#?}", ypred);

[
    Value(data=0.9131097886864816, grad=-0.17378042262703675 ),
    Value(data=-0.8918277596810039, grad=0.21634448063799216 ),
    Value(data=-0.9281754074968825, grad=0.14364918500623491 ),
    Value(data=0.9012427651510974, grad=-0.19751446969780528 ),
]
